In [2]:
import pandas as pd

/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/codespace/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
pd.__version__

'2.3.3'

In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [5]:
import sklearn
sklearn.__version__

'1.0.2'

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
# Number of columns
df.shape[1]

19

In [8]:
#calculate duration in minutes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
#count records before dropping outliers
original_count = df.shape[0]
#drop outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]
new_count = df.shape[0]


In [9]:
#percent remaining
(new_count/original_count)*100

97.78326020432945

In [10]:
# standard deviation of trip duration
df['duration'].std()


10.159586559178086

In [11]:
# Apply one-hot encoding to id columns
#df = pd.get_dummies(df, columns=['PULocationID', 'DOLocationID'],sparse=True)
# count updated columns
#df.shape[1]


In [12]:
from scipy.sparse import hstack
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from scipy import sparse
# 1. Use OneHotEncoder with sparse output
ohe = OneHotEncoder()

X_train = ohe.fit_transform(df[['PULocationID', 'DOLocationID']])
y_train = df['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)



LinearRegression()

In [15]:
from sklearn.metrics import mean_squared_error
y_pred_t = model.predict(X_train)
rmse = mean_squared_error(y_train, y_pred_t, squared=False)

print(f"Validation RMSE: {rmse:.4f}")

Validation RMSE: 7.9462


In [16]:
X_train.shape[1]

518

In [17]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')
#calculate duration in minutes
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
#count records before dropping outliers
original_count = df_val.shape[0]
#drop outliers
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
new_count = df_val.shape[0]
#percent remaining
(new_count/original_count)*100

97.69026103182483

In [18]:
df_val

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.50,1.0,0.5,1.28,0.00,1.0,26.78,2.5,0.00,15.216667
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.00,1.0,0.5,9.00,0.00,1.0,45.00,2.5,0.00,14.366667
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.0,28.69,2.0,N,132,261,2,70.00,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75,35.366667
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.0,1.10,1.0,N,161,163,1,9.30,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.00,8.966667
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.0,2.60,1.0,N,246,79,2,15.60,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.00,13.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007521,2,2024-02-29 23:20:17,2024-02-29 23:28:25,NaN,0.35,NaN,None,164,170,0,21.02,0.0,0.5,0.00,0.00,1.0,25.02,NaN,NaN,8.133333
3007522,2,2024-02-29 23:38:37,2024-02-29 23:49:37,NaN,1.99,NaN,None,249,45,0,18.78,0.0,0.5,0.00,0.00,1.0,22.78,NaN,NaN,11.000000
3007523,2,2024-02-29 23:14:40,2024-02-29 23:19:29,NaN,0.84,NaN,None,113,249,0,8.82,0.0,0.5,0.00,0.00,1.0,12.82,NaN,NaN,4.816667
3007524,1,2024-02-29 23:12:06,2024-02-29 23:21:53,NaN,0.00,NaN,None,164,107,0,14.37,0.0,0.5,0.00,0.00,1.0,18.37,NaN,NaN,9.783333


In [19]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(df[['PULocationID', 'DOLocationID']])
X_val = ohe.transform(df_val[['PULocationID', 'DOLocationID']])

y_val = df_val['duration'].values


In [20]:

y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f"Validation RMSE: {rmse:.4f}")

Validation RMSE: 8.1234
